# Data Investigation - Stations

### Key Take Aways:

- Summer gets warm and windy
- Winter gets cool and calmer
- Year round it is at least a little cloudy (rank 3 out of 8)
- On the rare occurance of rain, primarily in Winter (Early December through Early April), it is in very little volume

In [1]:
%matplotlib inline

import matplotlib
import numpy as np
from scipy import stats
import math
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob

import seaborn as sns
sns.set()

### Import Data

In [2]:
print('Loading Station Data...')

try:
    file = 'station_data_clean.csv'
    
    station_import = pd.read_csv('../junk.csv')
except:
    try:
        file_path_slug = '../../../datasets/bayareabikeshare/*_station_data.csv'
        file_list = glob(file_path_slug)

        station_import = pd.DataFrame()

        counter = 1
        chunks = []

        for file in file_list:
            for chunk in pd.read_csv(file, chunksize=10000, iterator=True):
                chunk.columns = ['station_id', 'name', 'lat', 'long', 'dockcount', 'landmark', 'installation']            
                chunks.append(chunk)
            print('\tFinished file! (%d of %d)' % (counter, len(file_list)))
            counter += 1

        station_import = pd.concat(chunks)
        print('Data Loaded Successfully!')
    except:
        print('oops... something went wrong importing the data :(')

Loading Station Data...
	Finished file! (1 of 4)
	Finished file! (2 of 4)
	Finished file! (3 of 4)
	Finished file! (4 of 4)
Data Loaded Successfully!


In [3]:
def label_zip(row):
    if row['landmark'] == 'San Francisco':
       return '94107'
    if row['landmark'] == 'Redwood City':
        return '94063'
    if row['landmark'] == 'Palo Alto':
        return '94301'
    if row['landmark'] == 'Mountain View':
        return '94041'
    if row['landmark'] == 'San Jose':
        return '95113'
    return '99999'

def make_lat_long(row):
    lat = row['lat']
    long = row['long']
    return (lat, long)

In [4]:
print('Station Data Cleanup Started...')

station_data = station_import.copy()

# remove dulplicates
# print('\tremove dulplicates')
# station_data.drop_duplicates(keep='first', inplace=True)
print('\tdropping empty rows')
station_data.dropna(how='all', inplace=True)

# set datatype for each column
print('\tset datatype for each column')
station_data['station_id']   = station_data['station_id'].astype('int')
station_data['name']         = station_data['name'].astype('str')
station_data['lat']          = station_data['lat'].astype('float')
station_data['long']         = station_data['long'].astype('float')
station_data['landmark']     = station_data['landmark'].astype('category')

# add a zipcode column for later comparison with weather data
station_data['zip_code'] = station_data.apply(lambda row: label_zip (row),axis=1)
# station_data['zip_code'] = station_data['landmark'].astype('str')

# create lat,lon tuple column
station_data['lat_long'] = station_data.apply(lambda row: make_lat_long (row),axis=1)

station_data.drop_duplicates(station_data.columns, keep='first', inplace=True)

# # reindex to remove some extra duplicate
# print('\tcorrecting index')
# station_data.reset_index(inplace=True)
# station_data.drop_duplicates(['station_id', 'installation'], keep='first', inplace=True)
# station_data.set_index('station_id', inplace=True)
# station_data.sort_index(inplace=True)

station_data.sort_values(by=['station_id', 'installation'], inplace=True)


# del station_data['index']
print('Cleaning complete!')


Station Data Cleanup Started...
	dropping empty rows
	set datatype for each column
Cleaning complete!


In [7]:
station_data[:50]

,station_id,name,lat,long,dockcount,landmark,installation,zip_code,lat_long
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27.0,San Jose,8/6/2013,95113,"(37.329732, -121.90178200000001)"
1,3,San Jose Civic Center,37.330698,-121.888979,15.0,San Jose,8/5/2013,95113,"(37.330698, -121.888979)"
2,4,Santa Clara at Almaden,37.333988,-121.894902,11.0,San Jose,8/6/2013,95113,"(37.333988, -121.894902)"
3,5,Adobe on Almaden,37.331415,-121.893200,19.0,San Jose,8/5/2013,95113,"(37.331415, -121.8932)"
4,6,San Pedro Square,37.336721,-121.894074,15.0,San Jose,8/7/2013,95113,"(37.336721000000004, -121.894074)"
5,7,Paseo de San Antonio,37.333798,-121.886943,15.0,San Jose,8/7/2013,95113,"(37.333798, -121.88694299999999)"
6,8,San Salvador at 1st,37.330165,-121.885831,15.0,San Jose,8/5/2013,95113,"(37.330165, -121.88583100000001)"
7,9,Japantown,37.348742,-121.894715,15.0,San Jose,8/5/2013,95113,"(37.348742, -121.89471499999999)"
8,10,San Jose City Hall,37.337391,-121.886995,15.0,San Jose,8/6/2013,95113,"(37.337391, -121.886995)"
9,11,MLK Library,37.335885,-121.885660,19.0,San Jose,8/6/2013,95113,"(37.335885, -121.88566000000002)"


In [ ]:
station_data.to_csv('../clean_data/station_data_clean.csv', encoding='utf-8')

In [ ]:
station_data.iloc[:50]

In [ ]:
station_data.iloc[50:]

In [ ]:
# bay area bike share did not become active until 8/23/13, 
# adjust any installation dates prior to then to 8/23/13

def adjust_to_activation_date(row):
    print(row['installation'])
    print(type(row['installation']))
    print()

    
    
    
    






In [ ]:
station_clean = station_data.copy()
station_clean['installation'] = pd.to_datetime(station_clean['installation'])
station_clean['installation'] = station_clean.apply(lambda row: adjust_to_activation_date(row),axis=1)